In [52]:
import json
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [61]:
movie_links = []
with open('./movie_links.jsonl', 'r', encoding='utf-8') as f:
  for line in f:
    data = json.loads(line.strip())
    for link in data:
      movie_links.append(link)
print(len(movie_links))
print(movie_links[4])


29650
https://www.imdb.com/title/tt13406094/?ref_=sr_i_5


In [62]:
driver = webdriver.Chrome()
for movie_link in tqdm(movie_links[4:]):
    movie = {
      'name': '',
      'src': movie_link,
      'releasedAt': '',
      'description': '',
      'poster': '',
      'seasons': 0,
      'genres': [],
      'countries': [],
      'productions': [],
      'trailer': '',
      'imdbRating': '',
      'casts': [],
      'episodes': []
    }

    episode = {
      'number': 1,
      'season': 1,
      'name': '',
      'description': '',
      'src': '',
      'thumbnail': '',
      'airedAt': '',
      'duration': '',
      'photos': []
    }

    driver.get(movie_link)

    try:
      movie_name = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//span[@data-testid="hero__primary-text"]'))
      )
      movie['name'] = movie_name.text
    except Exception as e:
      print(e)
      movie['name'] = ''

    try:
      movie_poster = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//div[@data-testid="hero-media__poster"]/div[1]/img'))
      )
      movie['poster'] = movie_poster.get_attribute('src')
    except Exception as e:
      print(e)
      movie['poster'] = ''

    try:
      movie_description = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//span[@data-testid="plot-l"]'))
      )
      movie['description'] = movie_description.text.strip()
    except Exception as e:
      print(e)
      movie['description'] = ''
    
    try:
      movie_genres = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//section[@data-testid="hero-parent"]'))
      ).find_element(By.CLASS_NAME, 'ipc-chip-list__scroller').find_elements(By.TAG_NAME, 'a')
      movie['genres'] = [genre.text.strip() for genre in movie_genres]
    except Exception as e:
      print(e)
      movie['genres'] = []

    movie_details = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.XPATH, '//div[@data-testid="title-details-section"]'))
    ).find_elements(By.XPATH, './ul')

    if len(movie_details) > 0:
      details = movie_details[0].find_elements(By.XPATH, './li')
      for detail in details:
        detail_name = detail.find_element(By.XPATH, './*[1]')

        if detail_name.tag_name == 'a' or detail_name.tag_name == 'span':
          category = detail_name.text.strip()
          if category == 'Release date':
            movie['releasedAt'] = detail.find_element(By.XPATH, './div').find_element(By.TAG_NAME, 'a').text.strip()
          elif category == 'Countries of origin' or category == 'Country of origin':
            movie['countries'] = [country.text.strip() for country in detail.find_element(By.XPATH, './div').find_elements(By.TAG_NAME, 'a')]
          elif category == 'Production companies' or category == 'Production company':
            movie['productions'] = [production.text.strip() for production in detail.find_element(By.XPATH, './div').find_elements(By.TAG_NAME, 'a')]  

    cast_section = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.XPATH, '//section[@data-testid="title-cast"]/div[@data-testid="shoveler"]'))
    )

    cast_items = cast_section.find_elements(By.XPATH, '//div[@data-testid="title-cast-item"]')
    for cast_item in cast_items:
      movie_cast = {
        'cast': {
          'name': '',
          'avatar': '',
          'src': ''
        },
        'role': '',
      }
      cast_avatar_section = cast_item.find_element(By.XPATH, './div[1]')
      try:
        cast_avatar = cast_avatar_section.find_element(By.TAG_NAME, 'img').get_attribute('src')
      except Exception as e:
        print(e)
        cast_avatar = ''

      try:
        cast_src = cast_avatar_section.find_element(By.TAG_NAME, 'a').get_attribute('href')
      except Exception as e:
        print(e)
        cast_src = ''

      try:
        cast_details_section = cast_item.find_element(By.XPATH, './div[2]').find_elements(By.TAG_NAME, 'a')
        cast_name = cast_details_section[0].text.strip()
        cast_role = cast_details_section[1].find_element(By.TAG_NAME, 'span').text.strip()
      except Exception as e:
        print(e)
      movie_cast['cast']['name'] = cast_name
      movie_cast['cast']['avatar'] = cast_avatar
      movie_cast['cast']['src'] = cast_src
      movie_cast['role'] = cast_role
      movie['casts'].append(movie_cast)

    episode_button = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.XPATH, '//section[@data-testid="episodes-widget"]'))
    ).find_element(By.TAG_NAME, 'a')
  
    driver.get(episode_button.get_attribute('href'))

    season_bar = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[1]/div[2]/ul'))
    )
    seasons = season_bar.find_elements(By.XPATH, './a')
    season_numbers = [int(season.text) for season in seasons]
    movie['seasons'] = len(season_numbers)
    season_links = [season.get_attribute('href') for season in seasons]
    episodes = []

    for season_number, season_link in zip(season_numbers, season_links):
      if season_number != 1: 
        driver.get(season_link)
      episode_section = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[2]'))
      )
      episode_divs = episode_section.find_elements(By.XPATH, './article')
      episode_links = [episode_div.find_element(By.TAG_NAME, 'a').get_attribute('href') for episode_div in episode_divs]
      episodes = []

      for episode_number, episode_link in enumerate(episode_links):
        driver.get(episode_link)
        try:
          episode_name = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[1]/h1/span'))
          )
          episode_name_value = episode_name.text.strip()
        except Exception as e:
          print(e)
          episode_name_value = ''

        try:
          aired_at = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[1]/ul/li[1]'))
          )
          aired_at_value = aired_at.text.strip().replace('Episode aired', '').strip()
        except Exception as e:
          print(e)
          aired_at_value = ''

        try:
          duration = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[1]/ul/li[3]'))
          )
          duration_value = duration.text.strip()
        except Exception as e:
          print(e)
          duration_value = ''
        
        try:
          description = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//p[@data-testid="plot"]'))
          ).find_elements(By.TAG_NAME, 'span')
          description_value = description[1].text.strip()
        except Exception as e:
          print(e)
          description_value = ''

        try:
          thumbnail = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[4]/div[1]/div/div/div[1]/img'))
          )
          thumbnail_value = thumbnail.get_attribute('src')
        except Exception as e:
          print(e)
          thumbnail_value = ''

        photo_link = WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.XPATH, '//section[@data-testid="Photos"]'))
        )
        photos_link_value = photo_link.find_element(By.TAG_NAME, 'a').get_attribute('href')
        photos = []
        total_photos = photo_link.find_element(By.TAG_NAME, 'h3').find_elements(By.TAG_NAME, 'span')
        if len(total_photos) > 0:
          total_photos_value = min(int(total_photos[0].text.strip()), 9)
          driver.get(photos_link_value)
          for photo_number in range(total_photos_value):
            try:
              media_viewer = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@data-testid="media-viewer"]'))
              )
              image = media_viewer.find_elements(By.XPATH, './div')[3].find_element(By.TAG_NAME, 'img')
              image_src = image.get_attribute('src')
            except Exception as e:
              print(e)
              image_src = ''

          
            if image_src:
              photos.append(image_src)
            next_image_button = WebDriverWait(driver, 10).until(
              EC.element_to_be_clickable((By.XPATH, '//div[@data-testid="media-viewer"]/div[3]'))
            )
            next_image_button.click()
            
        episode_info = {
          'number': episode_number + 1,
          'season': season_number,
          'name': episode_name_value,
          'description': description_value,
          'src': episode_link,
          'thumbnail': thumbnail_value,
          'airedAt': aired_at_value,
          'duration': duration_value,
          'photos': list(set(photos))
        }
        movie['episodes'].append(episode_info)

    with open('./movie_final_results.jsonl', 'a', encoding='utf-8') as f:
      f.write(json.dumps(movie) + '\n')


  0%|          | 0/29646 [00:00<?, ?it/s]

list index out of range
list index out of range
Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CD4402F5+28725]
	(No symbol) [0x00007FF7CD3A2AE0]
	(No symbol) [0x00007FF7CD23510A]
	(No symbol) [0x00007FF7CD2893D2]
	(No symbol) [0x00007FF7CD2895FC]
	(No symbol) [0x00007FF7CD2D3407]
	(No symbol) [0x00007FF7CD2AFFEF]
	(No symbol) [0x00007FF7CD2D0181]
	(No symbol) [0x00007FF7CD2AFD53]
	(No symbol) [0x00007FF7CD27A0E3]
	(No symbol) [0x00007FF7CD27B471]
	GetHandleVerifier [0x00007FF7CD76F30D+3366989]
	GetHandleVerifier [0x00007FF7CD7812F0+3440688]
	GetHandleVerifier [0x00007FF7CD7778FD+3401277]
	GetHandleVerifier [0x00007FF7CD50AAAB+858091]
	(No symbol) [0x00007FF7CD3AE74F]
	(No symbol) [0x00007FF7CD3AA304]
	(No symbol) [0x00007FF7CD3AA49D]
	(No symbol) [0x00007FF7CD398B69]
	BaseThreadInitThunk [0x00007FFA33C4E8D7+23]
	RtlUserThreadStart [0x00007FFA34FBFBCC+44]

Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CD4402F5+28725]
	(No symbol) [0x00007FF7CD3A2AE0]
	(No symbol) [0x00007FF7C

  0%|          | 1/29646 [04:29<2219:04:17, 269.48s/it]

Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CD4402F5+28725]
	(No symbol) [0x00007FF7CD3A2AE0]
	(No symbol) [0x00007FF7CD23510A]
	(No symbol) [0x00007FF7CD2893D2]
	(No symbol) [0x00007FF7CD2895FC]
	(No symbol) [0x00007FF7CD2D3407]
	(No symbol) [0x00007FF7CD2AFFEF]
	(No symbol) [0x00007FF7CD2D0181]
	(No symbol) [0x00007FF7CD2AFD53]
	(No symbol) [0x00007FF7CD27A0E3]
	(No symbol) [0x00007FF7CD27B471]
	GetHandleVerifier [0x00007FF7CD76F30D+3366989]
	GetHandleVerifier [0x00007FF7CD7812F0+3440688]
	GetHandleVerifier [0x00007FF7CD7778FD+3401277]
	GetHandleVerifier [0x00007FF7CD50AAAB+858091]
	(No symbol) [0x00007FF7CD3AE74F]
	(No symbol) [0x00007FF7CD3AA304]
	(No symbol) [0x00007FF7CD3AA49D]
	(No symbol) [0x00007FF7CD398B69]
	BaseThreadInitThunk [0x00007FFA33C4E8D7+23]
	RtlUserThreadStart [0x00007FFA34FBFBCC+44]

Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CD4402F5+28725]
	(No symbol) [0x00007FF7CD3A2AE0]
	(No symbol) [0x00007FF7CD23510A]
	(No symbol) [0x00007FF7CD2893D2]
	(No 

  0%|          | 2/29646 [08:44<2148:13:49, 260.88s/it]

...

Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CD4402F5+28725]
	(No symbol) [0x00007FF7CD3A2AE0]
	(No symbol) [0x00007FF7CD23510A]
	(No symbol) [0x00007FF7CD2893D2]
	(No symbol) [0x00007FF7CD2895FC]
	(No symbol) [0x00007FF7CD2D3407]
	(No symbol) [0x00007FF7CD2AFFEF]
	(No symbol) [0x00007FF7CD2D0181]
	(No symbol) [0x00007FF7CD2AFD53]
	(No symbol) [0x00007FF7CD27A0E3]
	(No symbol) [0x00007FF7CD27B471]
	GetHandleVerifier [0x00007FF7CD76F30D+3366989]
	GetHandleVerifier [0x00007FF7CD7812F0+3440688]
	GetHandleVerifier [0x00007FF7CD7778FD+3401277]
	GetHandleVerifier [0x00007FF7CD50AAAB+858091]
	(No symbol) [0x00007FF7CD3AE74F]
	(No symbol) [0x00007FF7CD3AA304]
	(No symbol) [0x00007FF7CD3AA49D]
	(No symbol) [0x00007FF7CD398B69]
	BaseThreadInitThunk [0x00007FFA33C4E8D7+23]
	RtlUserThreadStart [0x00007FFA34FBFBCC+44]

Message: no such element: Unable to locate element: {"method":"tag name","selector":"img"}
  (Session info: chrome=132.0.6834.160); For documentation on this error, please vis

  0%|          | 33/29646 [5:28:38<13117:58:20, 1594.73s/it]

Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CD4402F5+28725]
	(No symbol) [0x00007FF7CD3A2AE0]
	(No symbol) [0x00007FF7CD23510A]
	(No symbol) [0x00007FF7CD2893D2]
	(No symbol) [0x00007FF7CD2895FC]
	(No symbol) [0x00007FF7CD2D3407]
	(No symbol) [0x00007FF7CD2AFFEF]
	(No symbol) [0x00007FF7CD2D0181]
	(No symbol) [0x00007FF7CD2AFD53]
	(No symbol) [0x00007FF7CD27A0E3]
	(No symbol) [0x00007FF7CD27B471]
	GetHandleVerifier [0x00007FF7CD76F30D+3366989]
	GetHandleVerifier [0x00007FF7CD7812F0+3440688]
	GetHandleVerifier [0x00007FF7CD7778FD+3401277]
	GetHandleVerifier [0x00007FF7CD50AAAB+858091]
	(No symbol) [0x00007FF7CD3AE74F]
	(No symbol) [0x00007FF7CD3AA304]
	(No symbol) [0x00007FF7CD3AA49D]
	(No symbol) [0x00007FF7CD398B69]
	BaseThreadInitThunk [0x00007FFA33C4E8D7+23]
	RtlUserThreadStart [0x00007FFA34FBFBCC+44]

list index out of range


  0%|          | 33/29646 [5:28:44<4916:40:21, 597.71s/it]  


ValueError: invalid literal for int() with base 10: 'Unknown'